In [ ]:
!nvidia-smi

Sat Feb 20 05:00:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install flask-ngrok
!pip install flask-cors
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

questions = [
    "who is the best player in gsw?",
    "How many pretrained models are available in Transformers?",
    "What does Transformers provide?",
    "Transformers provides interoperability between which frameworks?",
]
print(len(text))
for question in questions:
    
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, padding=True, max_length=512, return_tensors="pt")
    offset_map = tokenizer.encode_plus(question, text, add_special_tokens=True, padding=True, max_length=512, return_tensors="pt", return_offsets_mapping=True)['offset_mapping'][0]
    input_ids = inputs["input_ids"].tolist()[0]

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    res = model(**inputs)
    answer_start_scores = res['start_logits']
    answer_end_scores = res['end_logits']
    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

    start = offset_map[answer_start][0]
    end = offset_map[answer_end][1]
    print(start, end)
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(text[start:end])
    print("\n")

363
tensor(335) tensor(17)
Question: who is the best player in gsw?
Answer: 



tensor(256) tensor(268)
Question: How many pretrained models are available in Transformers?
Answer: over 32 +
over 32+ pre


tensor(94) tensor(125)
Question: What does Transformers provide?
Answer: general - purpose architectures
general-purpose
architectures (


tensor(335) tensor(362)
Question: Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch
TensorFlow 2.0 and PyTorch.




In [ ]:
get_answer(questions, [text])

['over 32+ pre',
 'general-purpose\narchitectures (',
 'TensorFlow 2.0 and PyTorch.']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
def get_answer(questions, texts):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    answers = []
    for question in questions:
        for text in texts:

            inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, padding=True, max_length=512, return_tensors="pt").to(device)
            offset_map = tokenizer.encode_plus(question, text, add_special_tokens=True, padding=True, max_length=512, return_tensors="pt", return_offsets_mapping=True)['offset_mapping'][0]
            input_ids = inputs["input_ids"].tolist()[0]
            
            text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
            model.to(device)
            res = model(**inputs)
            model.to('cpu')
            answer_start_scores = res['start_logits']
            answer_end_scores = res['end_logits']
            answer_start = torch.argmax(answer_start_scores)
            answer_end = torch.argmax(answer_end_scores) + 1
            if answer_start == answer_end:
                continue

            start = offset_map[answer_start][0]
            end = offset_map[answer_end][1]
            # answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
            answer = text[start:end]
            if len(answer) == 0:
                continue
            answers.append(answer)
            if len(answers) > 10:
                return answers
    return answers

In [ ]:
from flask import Flask 
from flask import request
from flask_ngrok import run_with_ngrok
from flask_cors import CORS, cross_origin
from flask import jsonify



In [ ]:

app = Flask(__name__) 
# CORS(app)
run_with_ngrok(app)    
app.config['CORS_HEADERS'] = 'Content-Type'
cors = CORS(app, resources={r"/api/*": {"origins": "*"}})
@app.route("/") 
def home(): 
    return "<h1>GFG is great platform to learn</h1>"

# @cross_origin(headers=['Content-Type']) # Send Access-Control-Allow-Headers
@app.route('/api/query', methods = ['POST', 'GET'])
@cross_origin(headers=['Content-Type'])
def get_query_from_react():
    sep = '%$'
    data = request.get_json()
    print(data.keys())
    print(data['data'])
    
    texts = data['data'].split(sep)
    question = texts[0]
    contents = [text for text in texts[1:] if len(text.split(' ')) > 20]
    print("number of paragraphs: ", len(contents))
    print("Question is: ", question)
    print("Contents are: ", contents)
    answers = sep.join(get_answer([question], contents))
    print(answers)
    print("Answer: ", answers)
    #r = "the"
    return jsonify(answers)


In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://272678b23a91.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
dict_keys(['data'])
What is the declaration of independence?%$United States Declaration of Independence%$From Wikipedia, the free encyclopedia%$United States Declaration of Independence%$1823 facsimile of the engrossed copy%$Created	June–July 1776%$Ratified	July 4, 1776%$Location	Engrossed copy: National Archives Building Rough draft: Library of Congress%$Author(s)	Thomas Jefferson, Committee of Five%$Signatories	56 delegates to the Second Continental Congress%$Purpose	To announce and explain separation from Great Britain[1]%$This article is part of a series about the%$Declaration of Independence%$United States Constitution%$The United States Declaration of Independence (formally The unanimous Declaration of the thirteen united States of America) is the pronouncement adopted by the Second Continental Congress meeting in Philadelphia, Pennsylvania, on July 4, 1776. The Declaration explained wh

127.0.0.1 - - [20/Feb/2021 05:09:33] "POST /api/query HTTP/1.1" 200 -


the pronouncement adopted%$a formal explanation of why Congress had voted to declare independence from Great Britain,%$printed Dunlap broadside that%$The Declaration justified the independence of the United States by%$e truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain unalienable Rights, that among these are Life, Liberty and the pursuit of Happiness.%$statement of principles through which the United States Constitution should be interpreted.%$Declaration of United Belgian States issued%$the sentiments of America.%$the Thirteen Colonies and Great Britain had been at war for more than a year.%$"blows must decide whether they are to be subject to this country or independent".[%$no one yet advocated declaring it.[
Answer:  the pronouncement adopted%$a formal explanation of why Congress had voted to declare independence from Great Britain,%$printed Dunlap broadside that%$The Declaration justified the independence of the United S

...

Zoom + Piazza (%$Piazza forum, where you will also find all the announcements. For external enquiries, personal matters, or in emergencies, you can email us at cs329s-win2021-staff@lists.stanford.%$faculty.%$machine learning systems. The%$machine learning project%$machine learning production%$Machine learning systems%$tutorial approach%$computer science principles%$Stanford affiliates. External requests will be determined on a case by case basis, mostly because the course is hosted on Canvas and%$Our human resources are limited.
Answer:  Zoom + Piazza (%$Piazza forum, where you will also find all the announcements. For external enquiries, personal matters, or in emergencies, you can email us at cs329s-win2021-staff@lists.stanford.%$faculty.%$machine learning systems. The%$machine learning project%$machine learning production%$Machine learning systems%$tutorial approach%$computer science principles%$Stanford affiliates. External requests will be determined on a case by case basis, mostl

127.0.0.1 - - [20/Feb/2021 06:29:33] "POST /api/query HTTP/1.1" 200 -


the pronouncement adopted%$a formal explanation of why Congress had voted to declare independence from Great Britain,%$printed Dunlap broadside that%$The Declaration justified the independence of the United States by%$Liberty and the pursuit of Happiness.%$statement of principles through%$Declaration of United Belgian States issued%$the sentiments of America.%$the Thirteen Colonies and Great Britain had been at war for more than a year.%$blows must decide whether they are to be subject to this country or independent"%$no one yet advocated declaring it.[
Answer:  the pronouncement adopted%$a formal explanation of why Congress had voted to declare independence from Great Britain,%$printed Dunlap broadside that%$The Declaration justified the independence of the United States by%$Liberty and the pursuit of Happiness.%$statement of principles through%$Declaration of United Belgian States issued%$the sentiments of America.%$the Thirteen Colonies and Great Britain had been at war for more tha

127.0.0.1 - - [20/Feb/2021 06:31:34] "POST /api/query HTTP/1.1" 200 -


The United States Declaration of Independence (formally The unanimous Declaration of the thirteen united States of America) is the pronouncement adopted by the Second Continental Congress meeting in Philadelphia, Pennsylvania, on July 4, 1776. The Declaration explained why the Thirteen Colonies at war with the Kingdom of Great Britain regarded themselves as thirteen independent sovereign states, no longer under British rule.%$a formal explanation of why Congress had voted to declare independence from Great Britain,%$printed Dunlap broadside that%$The Declaration justified the independence of the United States by listing 27 colonial grievances against King George III and by asserting certain natural and legal rights, including a right of revolution.%$a statement of principles through which the United States Constitution should be interpreted.%$Declaration of United Belgian States issued%$I will cease to exist before I yield to a connection on such terms as the British Parliament propose